In [ ]:
import numpy as np
import random
import math
#capital because it is static
ROW_COUNT = 6
COLUMN_COUNT = 7
# 
player=0
comp=1

EMPTY=0

PLAYER_PIECE=1
COMP_PIECE=2

WINDOW_LENGHT=4
play =str(input("player VS player (write(p)) or player VS copmuter (write(c))"))
#to creat board
def create_board():
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))

    return board
#to put piece 
def drop_piece(board,row,col,piece):
    board[row][col]= piece

#to put in correct location 
def is_valid_location(board,col):
    #if this condition is true we will let the use drop piece here.
    #if not true that means the col is not vacant
    #row count-1=5 that mean at the at the last row this cells of col we choice 
    return board[ROW_COUNT-1][col]==0
#we search at another row 
def get_next_open_row(board,col):
    for r in range(ROW_COUNT):
        if board[r][col]==0:
            return r

def print_board(board):
    #flip reverse all the array because that zero row at top and last row at last 
    print(np.flip(board,0))

def winning_move(board, piece):
    # Check horizontal locations for win
    #column_count-3 because after index 2 that we cant make 4
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True
 
    # Check vertical locations for win
    for c in range(COLUMN_COUNT):
        #row_count-3 because after that at indx 3 we cant make 4
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True
 
    # Check positively sloped diaganols down to up
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True
 
    # Check negatively sloped diaganolstop to down
    # 0 to 4
    # 3 to 6
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
            
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score            
            
            
#to know how row is empty in board
def score_position(board, piece):
    #score horizontal
    score=0
    ## Score center column
    #col//2 will get resutl division int not float
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    center_count = center_array.count(piece)
    score += center_count * 3
    
    
    for r in range(ROW_COUNT):
        #we need a specific row this forloop from i to specific row and all column
        row_array=[int(i) for i in list(board[r,:])]
        #we need to count in one row how many we can get 4 mark the same we found it was four time
        #this for loop to from 0 to col index 3 after that we cant get 4
        for c in range(COLUMN_COUNT -3):
            window=row_array[c:c+WINDOW_LENGHT]
            score += evaluate_window(window, piece)
        
    
    # Score vertical
    #we need a specific COL this forloop from i to specific COL and all ROW
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            window = col_array[r:r+WINDOW_LENGHT]
            score += evaluate_window(window, piece)
       
    
     # Score posiive sloped diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGHT)]
            score += evaluate_window(window, piece)
        

    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGHT)]
            score += evaluate_window(window, piece)
    return score
                                     
def is_terminal_node(board):
    return winning_move(board,PLAYER_PIECE) or winning_move(board,COMP_PIECE) or len(get_valid_locations(board)) == 0
                        
def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, COMP_PIECE):
                return  (None ,100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None,-10000000000000)
            else :#game is over
                return (None,0)
        else: # Depth is zero
            return (None , score_position(board, COMP_PIECE))   
    
    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, COMP_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column ,value

    else: # Minimizing player
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

    
#in first row    
def get_valid_locations(board):
    valid_locations=[]
    for col in range(COLUMN_COUNT):
        if is_valid_location(board , col):
            valid_locations.append(col)
    return valid_locations        
        
#        
def pick_best_move(board , piece):    
    valid_locations = get_valid_locations(board)
    #best score will take negative num because if we get negative number dont make error
    best_score=-1000
    best_col=random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board , col)
        temp_board = board.copy()
        drop_piece(temp_board, row, col, piece)
        score= score_position(temp_board ,piece)        
        if score > best_score:
            best_score=score
            best_col=col
            
    return best_col

board = create_board()
print_board(board)
game_over = False

#random who begain first
turn=player

while not game_over:
    
    #Ask for player 1 input
    if turn == player:
        #int because col type is str
        col = int(input("Player 1, Make your Selection(0-6):"))
        #Player 1 will drop a piece on the board
        if is_valid_location(board,col):
            row = get_next_open_row(board,col)
            drop_piece(board,row,col,PLAYER_PIECE)
            
            if winning_move(board,PLAYER_PIECE):
                print('player 1 winnes')
                game_over=True
                
            turn += 1
            turn = turn % 2
            
            print_board(board)
       
        # Ask for Player 2 Input
        if play== 'p': 
            col = int(input("Player 2, Make your Selection(0-6):"))

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, 2)
                    
                if winning_move(board, 2):
                        print("Player 2 wins!!")
                        game_over = True
        else:           
             #Ask for comp  input
            if turn==comp and not game_over:
                #between 0 to 6
                #col= pick_best_move(board ,COMP_PIECE)
                col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

                #Player 2 will drop a piece on the board
                if is_valid_location(board,col):
                    row = get_next_open_row(board,col)
                    drop_piece(board,row,col,COMP_PIECE)
 
                    if winning_move(board,COMP_PIECE):
                         print('player 2 winnes')
                         game_over = True  
                        
    
    print_board(board)
             
    turn += 1
    turn = turn % 2

player VS player (write(p)) or player VS copmuter (write(c))c
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
